# Analysis of COMD coefficient across individuals

In [ ]:
from data_utils.data_handling import DataHandler
import pandas as pd
import numpy as np
from plotnine import *

In [ ]:
data_path = '../../data/'

Let's get the ei ratios

We will have to filter the genes by expressed only per tissue and select the major transcripts afterwards

In [ ]:
ei_ratios_df = pd.read_csv(data_path+'EI-ratios-masked.csv')

In [ ]:
ei_ratios_df = ei_ratios_df.rename({'gene.id':'gene_id'}, axis=1).set_index('gene_id')
ei_ratios_df

Let's load the tissue annotations of the samples from gtex:

In [ ]:
gtex_annotation = pd.read_csv(data_path+'bam_files_condition-annotation.csv')
gtex_annotation

In [ ]:
ei_ratios_df = ei_ratios_df.T.merge(gtex_annotation.loc[:, ['sample.id', 'subtissue']], left_index=True, right_on='sample.id')
ei_ratios_df

In [ ]:
ei_ratios_df['subtissue'] = ei_ratios_df['subtissue'].str.replace('_', ' ')

Reading major isoforms table:

In [ ]:
major_iso_df = pd.read_csv(data_path+'gtex_major_isoform_per_subtissue.csv').set_index('gene_id')
major_iso_df

In [ ]:
major_iso_df.columns = major_iso_df.columns.str.replace(' - ', ' ')
major_iso_df.columns = major_iso_df.columns.str.replace('-', ' ')
major_iso_df.columns = major_iso_df.columns.str.replace('(', '')
major_iso_df.columns = major_iso_df.columns.str.replace(')', '')
major_iso_df

In [ ]:
samples_ei_df_melted = ei_ratios_df.melt(id_vars=['sample.id', 'subtissue'], var_name='gene_id', value_name='ei_ratio')
samples_ei_df_melted

Let's melt the major isoforms table so we can merge it with the samples

In [ ]:
melted_major_isoforms = pd.melt(major_iso_df.reset_index(), id_vars='gene_id', value_name='major_isoform', var_name='subtissue')
melted_major_isoforms = melted_major_isoforms.dropna()
melted_major_isoforms

In [ ]:
samples_mi_ei_df = samples_ei_df_melted.merge(melted_major_isoforms, left_on=['gene_id', 'subtissue'], right_on=['gene_id', 'subtissue'], how='left')
samples_mi_ei_df

Drop nas (which corrrespond to non expressed genes)

In [ ]:
samples_mi_ei_df = samples_mi_ei_df.dropna()
samples_mi_ei_df

In [ ]:
major_iso_df = samples_mi_ei_df.pivot(values='ei_ratio', columns='sample.id', index='major_isoform')
major_iso_df

In [ ]:
thresh = int((2/3) * len(major_iso_df.columns)) #keep transcripts which are expressed in more than 2/3s of the samples
thresh

In [ ]:
ei_iso_df_non_centered_t = major_iso_df.dropna(thresh=thresh)
ei_iso_df_non_centered_t

In [ ]:
ei_iso_df_t = ei_iso_df_non_centered_t.sub(ei_iso_df_non_centered_t.mean(axis=1), axis=0)
ei_iso_df_t

## Load reference decoding rate

Let's load the gencode 19 decoding estimates and the CENTERED EI ratios

In [ ]:
dec_rates_df = pd.read_csv(data_path + 'gencode19_avg_dec_rate', sep=' ', names=['transcript_id','mtdr'], index_col=0)
dec_rates_df

In [ ]:
(ggplot(dec_rates_df, aes('mtdr'))
    + geom_histogram())

In [ ]:
ei_dec_df = ei_iso_df_t.join(dec_rates_df, how='inner')
ei_dec_df

## Load individual metadata

In [ ]:
gtex_annotation['individual_id'] = gtex_annotation['sample.name'].apply(lambda name: name.split('-')[0] + '-' + name.split('-')[1])
gtex_annotation['individual_id']

Number of individuals:

In [ ]:
len(gtex_annotation['individual_id'].unique())

In [ ]:
individual_metadata = pd.read_csv(data_path+'GTEX_Annotations_subjects', index_col=0)
individual_metadata

In [ ]:
individual_metadata = individual_metadata.merge(gtex_annotation.loc[:, ['individual_id', 'sample.name', 'sample.id']], left_on='SUBJID', right_on='individual_id')

In [ ]:
individual_metadata.set_index('sample.name', inplace=True)
individual_metadata

In [ ]:
all_atributes = pd.read_csv(data_path+'GTEx_Annotations.csv')

In [ ]:
all_atributes

All public metadata with bam file ids:

In [ ]:
all_metadata_df = individual_metadata.merge(all_atributes, left_index=True, right_on='SAMPID')
all_metadata_df

In [ ]:
#all_metadata_df.to_csv(data_path+'gtex_sample_metadata_public.csv')

## Compute the COMD coefficient per (indivdual, tissue) pair or sample

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score
from scipy.stats import spearmanr, pearsonr

In [ ]:
def linear_reg_sample(ei_sample_dec_time_df, sample, ind_var='mtdr'):

    model = LinearRegression()
    sample_ei = ei_sample_dec_time_df.loc[:, [sample, ind_var]].dropna()
    spearman_sample = spearmanr(sample_ei[ind_var], sample_ei[sample])[0]
    
    model.fit(X=sample_ei[ind_var].values.reshape(-1, 1), y=sample_ei[sample].values)
    
    exp_var = explained_variance_score(sample_ei[sample].values, 
                         model.predict(sample_ei[ind_var].values.reshape(-1, 1)))

    
    pearson_ei_dec = pearsonr(sample_ei[ind_var].values, sample_ei[sample].values)[0]
    
    coef = model.coef_[0]
    
    return coef, exp_var, pearson_ei_dec, spearman_sample

In [ ]:
sample = 'SRR1068687'

In [ ]:
linear_reg_sample(ei_dec_df, sample)

In [ ]:
lin_reg_dict = {'sample': [], 'comd_coef': [], 'exp_var': [], 'pearson_ei_dec': [], 'rho_sample_ei_dec': []}

for sample in ei_iso_df_t.columns:
    
    coef, exp_var, pearson, spearman_sample = linear_reg_sample(ei_dec_df, sample)
    lin_reg_dict['sample'].append(sample)
    lin_reg_dict['comd_coef'].append(coef)
    lin_reg_dict['exp_var'].append(exp_var)
    lin_reg_dict['pearson_ei_dec'].append(pearson)
    lin_reg_dict['rho_sample_ei_dec'].append(spearman_sample)
    
lin_reg_df = pd.DataFrame(lin_reg_dict)
lin_reg_df    

In [ ]:
lin_reg_df = lin_reg_df.sort_values('comd_coef')
lin_reg_df['sample'] = pd.Categorical(values=lin_reg_df['sample'], categories=lin_reg_df['sample'], ordered=True)

In [ ]:
(ggplot(lin_reg_df, aes('comd_coef'))
    + geom_histogram())

In [ ]:
comd_coef_samples_df = lin_reg_df.loc[:, ['sample', 'comd_coef']].set_index('sample')
comd_coef_samples_df

In [ ]:
comd_coef_samples_df.to_csv(data_path+'comd_coef_all_gtex_samples.csv')

In [ ]:
#comd_coef_samples_df = pd.read_csv(data_path + 'comd_coef_all_gtex_samples.csv', index_col=0)
#comd_coef_samples_df

In [ ]:
comd_coef_metadata_df = comd_coef_samples_df.merge(all_metadata_df, left_index=True, right_on='sample.id')
comd_coef_metadata_df

Saving exon/intron ratio for transcripts with CDS where dec rate was calculated

In [ ]:
ei_iso_df_t.loc[dec_rates_df.index.intersection(ei_iso_df_t.index)].T.join(comd_coef_metadata_df.set_index(
    'sample.id').loc[:, ['SMTS','comd_coef']], how='inner').rename({'SMTS':'tissue'}, axis=1).to_csv(
    '../../figures/figure_data/fig1/samples_ei_ratios_major_iso_centered_2_3_non_na_comd_coef.csv')

# Association between the COMD coefficient and sample attributes - GTEx private dataset

In [ ]:
from scipy.stats import spearmanr

def cor_and_plot(df, x, y, xlab=None, ylab=None):
    
    xlab = x if xlab is None else xlab
    ylab = y if ylab is None else ylab
    
    df_plot = df.loc[:, [x,y]].dropna()
    print(pearsonr(df_plot[x], df_plot[y]))
    
    
    p = (ggplot(df, aes(x, y))
        + geom_bin2d()
        + labs(x=xlab, y=ylab))
    
    return p

In [ ]:
sample_data_private = pd.read_csv(data_path+'GTEx_private_data.csv', index_col=0)
sample_data_private

In [ ]:
ann_comd_coef_df = comd_coef_metadata_df.merge(sample_data_private, left_on='SUBJID', right_on='submitter_id')
ann_comd_coef_df

In [ ]:
comd_coef_ann_df = ann_comd_coef_df.set_index('SUBJID')
comd_coef_ann_df

"SMTSISCH" -> ischemic time
"SMTSD" -> subtissue
"age_value" -> age
"sex" -> sex

## COMD coefficient vs ischemic time per age and tissue

In [ ]:
isc_age_df = ann_comd_coef_df.loc[:, ['SMTSISCH', 'SMTSD', 'SMTS','comd_coef','age_value', 'AGE']].dropna()

In [ ]:
isc_age_df

Some samples (blood) were extracted pre-mortem and their ischemic time is negative, it will not be considered.

In [ ]:
isc_age_df = isc_age_df[isc_age_df['SMTSISCH']>0].copy()

In [ ]:
isc_age_df['ischemic_log'] = np.log(isc_age_df['SMTSISCH'])

In [ ]:
tissues = isc_age_df['SMTS'].unique()

In [ ]:
isc_age_df['comd_coef_fc']=2**isc_age_df['comd_coef']

In [ ]:
isc_age_df.to_csv('../../figures/figure_data/fig3/comd_coef_isch_all_samples.csv')

In [ ]:
tissue='Esophagus'
(ggplot(isc_age_df[isc_age_df['SMTS']==tissue], aes('SMTSISCH', 'comd_coef_fc'))
        + geom_point()
        + scale_x_log10()
        + scale_y_log10(breaks=[0.5, 1, 1.5, 2], limits=(0.5,2))
        + geom_smooth(method='lm')
        + theme_bw()
        + theme(figure_size=(10, 7))
        + facet_wrap('AGE')
        + labs(x='Ischemic time [min]', y='COMD coefficient (FC/codon/s)',title='COMD coefficient vs Ischemic time in Esophagus samples per age group'))

In [ ]:
tissue='Esophagus'
(ggplot(isc_age_df[isc_age_df['SMTS']==tissue], aes('SMTSISCH', 'comd_coef_fc'))
        + geom_point()
        + scale_x_log10()
        + scale_y_log10(breaks=[0.5, 1, 1.5, 2], limits=(0.5,2))
        + geom_smooth(method='lm')
        + theme_bw()
        + theme(figure_size=(10, 7))
        + facet_wrap('AGE')
        + labs(x='Ischemic time [min]', y='COMD coefficient (FC/codon/s)',title='COMD coefficient vs Ischemic time in Esophagus samples per age group'))

In [ ]:
df = isc_age_df[(isc_age_df['SMTS']==tissue) & (isc_age_df['AGE']=='20-29')]
spearmanr(df['SMTSISCH'], df['comd_coef'])

## COMD coefficient vs Age

In [ ]:
isc_age_df

In [ ]:
(ggplot(isc_age_df, aes('SMTSISCH', 'comd_coef'))
    + geom_bin2d(bins=40)
    + geom_smooth(method='lm')
    + scale_x_log10()
    + theme_bw()
    + theme( panel_grid_major = element_blank(),
                    #panel_grid_minor = element_blank(),
                    #panel_border = element_blank(),
                    panel_background = element_blank(),
                axis_line=element_line())
    + labs(x='Ischemic time [min]', y='COMD coefficient'))

## Ischemic time association correcting for tissue and age

In [ ]:
import statsmodels.formula.api as smf

In [ ]:
corr_df = ann_comd_coef_df.loc[ann_comd_coef_df['SMTSISCH']>0, ['SMTSISCH', 'comd_coef', 'age_value', 'SMTSD','SUBJID']].drop_duplicates()
corr_df['isch_log'] = np.log(corr_df['SMTSISCH'])
corr_df

In [ ]:
def get_corrected_var(df, var_to_correct, covariates):
    
    formula = var_to_correct + '~' + ' + '.join(covariates)
    model = smf.ols(formula=formula, data=df)
    results=model.fit()
    
    return results.resid

In [ ]:
corr_df['comd_coef_res'] = get_corrected_var(corr_df, var_to_correct = 'comd_coef', covariates = ['age_value', 'SMTSD'])
corr_df['comd_coef_res']

In [ ]:
corr_df.to_csv('../../figures/figure_data/fig3/corrected_comd_coef_vs_ischemic_time.csv')

In [ ]:
(ggplot(corr_df, aes('SMTSISCH', 'comd_coef_res'))
    + geom_bin2d(bins=40)
    + geom_smooth(method='lm')
    + theme_bw()
    + scale_x_log10()
    + theme( panel_grid_major = element_blank(),
                    #panel_grid_minor = element_blank(),
                    #panel_border = element_blank(),
                    panel_background = element_blank(),
                axis_line=element_line())
    + labs(x='Ischemic time [min]', y='comd_coef*'))

In [ ]:
spearmanr(corr_df['SMTSISCH'], corr_df['comd_coef_res'])

## Individual specific COMD coefficient vs age

We can fit a model comd_coef ~ individual + tissue, and then get the individual parameter in order to have the individual specific COMD coefficient

In [ ]:
reference_individual = 'GTEX-WHSE'

In [ ]:
formula = f"comd_coef ~ C(SUBJID, Treatment(reference='{reference_individual}')) + SMTSD"

In [ ]:
ann_comd_coef_df

In [ ]:
model = smf.ols(formula=formula, data=ann_comd_coef_df)
results=model.fit()
results.summary()

In [ ]:
model_params = results.params.copy()
model_params.name='effect_on_comd_coef'
model_params.index.name='param'
model_params_df = model_params.to_frame().reset_index()
model_params_df

In [ ]:
model_params_df['param']=model_params_df.param.apply(lambda el: el.split('T.')[1][:-1] if 'T.' in el else el)

In [ ]:
model_params_df

In [ ]:
ind_comd_coef_df = ann_comd_coef_df.merge(model_params_df, left_on='SUBJID', right_on='param').loc[:, ['SUBJID','age_value','effect_on_comd_coef']].drop_duplicates()
ind_comd_coef_df

In [ ]:
#center effect_on_comd_coef
ind_comd_coef_df['effect_on_comd_coef'] = ind_comd_coef_df['effect_on_comd_coef'].sub(ind_comd_coef_df['effect_on_comd_coef'].mean())

In [ ]:
ind_comd_coef_df.to_csv('../../figures/figure_data/fig2/age_comd_coef_individual.csv')

In [ ]:
(ggplot(ind_comd_coef_df, aes('age_value', 'effect_on_comd_coef'))
    + geom_point(color='darkblue', alpha=0.35)
    + theme_bw()
    + geom_smooth(method='lm', color='blue')
 #+ geom_smooth(color='blue')
    + theme( #panel_grid_major = element_blank(),
                    panel_grid_minor = element_blank(),
                    #panel_border = element_blank(),
                    panel_background = element_blank()
           )
    + labs(x='individual\'s age', y='individual\'s COMD coefficient'))

In [ ]:
spearmanr(ind_comd_coef_df['age_value'], ind_comd_coef_df['effect_on_comd_coef'])

## Pathways associated with changes in COMD coefficient across individuals in the same tissue

In [ ]:
gene_tpm = pd.read_csv('https://storage.googleapis.com/gtex_analysis_v7/rna_seq_data/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct.gz', sep='\t', header=2).set_index('Description').drop('Name', axis=1).T
gene_tpm

In [ ]:
gene_tpm = gene_tpm.T

In [ ]:
gene_tpm = gene_tpm[gene_tpm>1]
gene_tpm

Only genes with at least 2/3 of non na across samples are considered

In [ ]:
gene_tpm = gene_tpm.dropna(thresh=int((2/3) * len(gene_tpm.columns)))
gene_tpm

Some genes appear multiple times for the same sample

In [ ]:
duplicated_genes = gene_tpm.loc[gene_tpm.index.duplicated()].index.unique()

In [ ]:
duplicated_genes

Remove duplicated genes:

In [ ]:
gene_tpm = gene_tpm.loc[gene_tpm.index.difference(duplicated_genes)]
gene_tpm

In [ ]:
genes = gene_tpm.index

In [ ]:
gene_tpm = comd_coef_metadata_df.merge(gene_tpm.T, left_on='SAMPID', right_index=True, suffixes=('', '_y'))
gene_tpm

In [ ]:
adjust_text_dict = {
    'expand_points': (0, 0),
}

tissues_of_interest=['Brain - Cerebellar Hemisphere', 'Skin - Not Sun Exposed (Suprapubic)','Thyroid', 'Stomach','Cells - EBV-transformed lymphocytes','Heart - Left Ventricle']

(ggplot(gene_tpm[gene_tpm['SMTSD'].isin(tissues_of_interest)], aes('NDUFB3', 'comd_coef', color='SMTSD'))
    + geom_point(size=1)
    + geom_smooth(method='lm')
    + theme(figure_size=(10, 10))
    + scale_x_log10())

In [ ]:
gene_tpm.loc[:, ['comd_coef', 'SMTSD', 'SMTS', 'NDUFB3']].to_csv('../../figures/figure_data/fig2/comd_coef_ndufb3_within_tissues.csv')

In [ ]:
adjust_text_dict = {
    'expand_points': (0, 0),
}


(ggplot(gene_tpm, aes( 'comd_coef', 'MT-RNR2'))
    + geom_bin2d(bins=60)
    + geom_smooth(method='lm')
    + theme(figure_size=(10, 10))
    + scale_y_log10())

In [ ]:
tissues=gene_tpm['SMTS'].unique()
tissues

## GSEA all samples

In [ ]:
def cor_gene_comd_coef(tissue_tpm_comd_coef_df, gene, na_thresh):
    if tissue_tpm_comd_coef_df[gene].isna().sum()>na_thresh:
        return np.nan
    else:
        df = tissue_tpm_comd_coef_df.loc[:, ['comd_coef', gene]].dropna()
        return spearmanr(df['comd_coef'], df[gene])[0]

# na_thresh_ratio, proportion of non na values for the gene to be considered in the correlation
def cor_all_genes_alpha(tissue_tpm_comd_coef_df, genes, na_thresh_ratio=2/3):
    
    na_thresh = na_thresh_ratio*len(tissue_tpm_comd_coef_df)
    cor_dict = {'gene':genes,'rho':[]}
    for gene in genes:
        cor_dict['rho'].append(cor_gene_comd_coef(tissue_tpm_comd_coef_df, gene, na_thresh))
        
    return pd.DataFrame(cor_dict)

#### Ex. Brain

In [ ]:
tissue_tpm_comd_coef_df = gene_tpm[gene_tpm['SMTS']=='Brain'].copy()
tissue_tpm_comd_coef_df

In [ ]:
cor_df = cor_all_genes_alpha(tissue_tpm_comd_coef_df, genes)
cor_df

In [ ]:
cor_df = cor_df.dropna().reset_index(drop=True)
cor_df

In [ ]:
ranked_df = cor_df.sort_values('rho', ascending=False).copy()

In [ ]:
(ggplot(ranked_df, aes('rho'))
    + geom_histogram())

In [ ]:
ranked_df.columns=[0,1]
ranked_df

In [ ]:
ranked_df.reset_index(drop=True, inplace=True)

In [ ]:
import gseapy as gp

In [ ]:
pre_res = gp.prerank(rnk=ranked_df, gene_sets='GO_Biological_Process_2018', 
                     processes=40,
                     permutation_num=1000, seed=123)

In [ ]:
pre_res.res2d.sort_values(by='NES')

In [ ]:
from gseapy.plot import gseaplot

term = 'respiratory electron transport chain (GO:0022904)'
gseaplot(rank_metric=pre_res.ranking, term=term, 
         **pre_res.results[term])

#### All samples

In [ ]:
pre_res_list =[]
for tissue in tissues:
    print(tissue)
    tissue_tpm_comd_coef_df = gene_tpm[gene_tpm['SMTS']==tissue]
    cor_tissue_df = cor_all_genes_alpha(tissue_tpm_comd_coef_df, genes)
    cor_tissue_df

    cor_tissue_df = cor_tissue_df.dropna().sort_values(by='rho')
    cor_tissue_df

    cor_tissue_df = cor_tissue_df.reset_index(drop=True)
    ranked_df = cor_tissue_df.copy()
    ranked_df.columns=[0,1]

    try:
        pre_res = gp.prerank(rnk=ranked_df, gene_sets='GO_Biological_Process_2018', 
                             processes=20,
                             permutation_num=1000, seed=123)
    except:
        continue
    pre_res.res2d['tissue']=tissue
    pre_res_list.append(pre_res.res2d)

In [ ]:
fdr_thresh=0.01
pre_res_filtered_list = []

for pre_res in pre_res_list:
    pre_res_filtered = pre_res[pre_res['FDR q-val']<=fdr_thresh].sort_values(by='NES')
    if(len(pre_res_filtered) == 0):
        continue
    #Select top and bottom n pathways 
    #pre_res_filtered = pre_res_filtered.iloc[np.r_[:n, len(pre_res_filtered)-n:len(pre_res_filtered)],:].drop_duplicates()
    pre_res_filtered_list.append(pre_res_filtered)

In [ ]:
pre_res_all_df = pd.concat(pre_res_filtered_list, join='outer').reset_index(drop=True)
pre_res_all_df

In [ ]:
pre_res_all_df.drop('Name', axis=1).to_csv(data_path + 'gsea_within_tissues_human_comd_coef.csv')